In [3]:
'''Build a dataset from the MarketplaceAI Production or test database'''
# encoding=utf8
import pandas as pd
import numpy as np
import pymongo
from bson.objectid import ObjectId
from datetime import datetime
import time
import csv


In [2]:
# !pip install pymongo 

     |████████████████████████████████| 419kB 8.7MB/s eta 0:00:01


In [18]:
def open_db(ip='localhost', port=27017):
    '''opens a mongodb client and
    returns the database merchants collection'''
    client = pymongo.MongoClient(ip, port, 
                                 username ='mktpl_user', 
                                 password ='ozvw0JV2ChW9Mu29',
                                authSource ='mktpl_db',
                                authMechanism = 'SCRAM-SHA-1')
    
    return client.mktpl_db.listings_db

def item_to_dict(item):
    '''Item found into data dictionary'''
    tags = item['processedData'].get('prod_tags_txt', '')
    tag = tags.split(',')[-1]

    # get the category
    category = item.get('alert_category', ['LEGIT'])
    if len(category) == 0:
        category = ['LEGIT']

    # Transform seller join date into a datetime
    #seller_join_date = item['processedData'].get('seller_join_date', None),
    #seller_jdate = datetime.strptime(seller_join_date[0],'%Y-%m-%d %H:%M:%S')
    
    # Identify customer
    customer = item['customer']
    if customer == ObjectId('58b65bb33f6f676a2b33fdd2'):
        customer = '11street'
    else:
        customer = '11street_poc'

    data = {'_id': item['_id'],
            'prod_id': item['processedData'].get('prod_id', ''),
            'prod_name_txt': item['processedData'].get('prod_name_txt', ''),
            'prod_price': float(item['processedData'].get('prod_price',
                                                          np.NaN)),
            'prod_price_currency': item['processedData'].get('prod_price_currency',
                                                             None),
            'prod_tags_txt': tags,
            'tag': tag,
            'prod_ship_loc': item['processedData'].get('prod_ship_loc', None),
            'seller_id': item['processedData'].get('seller_id', ''),
            'seller_email': item['processedData'].get('seller_email', None),
            #'seller_join_date': seller_jdate,
            'seller_loc_coarse': item['processedData'].get('seller_loc_coarse',
                                                           None),
            'seller_join_ipv4': item['processedData'].get('seller_join_ipv4',
                                                          None),
            'seller_list_ipv4': item['processedData'].get('seller_list_ipv4',
                                                          None),
            'created_datetime': item['created_datetime'],
            'modified_datetime': item.get('modified_datetime', None),
            'sent_datetime': item.get('sent_datetime', None),
            'category': category[0],
            'customer': customer
            }
    return data


In [22]:
import dateutil 

# Open the db connection
# listings_db = open_db('52.221.160.141')
listings_db = open_db('10.8.1.208')

# Save the results to a csv file
path = 'fraudwall/listing_1M.csv'

# query the database
# Select where customer is either
# '11street', or '11street_poc'
cursor = listings_db.find(
    {
        "customer": ObjectId('58b65bb33f6f676a2b33fdd2'),
        "is_false_positive": None, # {"$ne": None}
        "alert_category": [], # {"$ne": []}
        "sent_datetime": {"$gte": dateutil.parser.parse('2016-01-01')}, # gte = gtreater than equals to 
        
                             #ObjectId('57fc78f6632dd164144c022a')]}
    }).sort("_id", -1).limit(10)

# Column names in csv file
columns = ['_id', 'prod_id',
           'prod_name_txt', 'prod_price', 'prod_price_currency',
           'prod_tags_txt', 'tag', 'prod_ship_loc',
           'seller_id', 'seller_email',
           'seller_join_date', 'seller_loc_coarse',
           'seller_join_ipv4', 'seller_list_ipv4',
           'created_datetime', 'modified_datetime',
           'sent_datetime',
           'category', 'customer']


# Create a dataframe to store results
# df = pd.DataFrame(columns=columns)

# For each listing, extract information
# Anc write it to e csv file
with open(path, 'w+') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columns)
    writer.writeheader()
    start = time.time()
    for idx, item in enumerate(cursor):
        # Extract the data from the item
        # print(item)
        # import pdb; pdb.set_trace()
        data = item_to_dict(item)
        # Write to the csv file
        writer.writerow(data)
        # Add listing to dataframe
        # df = df.append(data, ignore_index=True)
        # Print every 1000 items
        if (idx % 1000) == 0:
            end = time.time()
            print(idx, end - start)
            start = time.time()

# Save the data to the csv file
# df.to_csv(path, columns=columns, encoding='utf-8')

0 0.027071714401245117


In [24]:
data

{'_id': ObjectId('5cf7d0885c98c110a1a9b724'),
 'category': 'LEGIT',
 'created_datetime': datetime.datetime(2019, 6, 5, 14, 24, 8, 806000),
 'customer': '11street',
 'modified_datetime': datetime.datetime(2019, 6, 5, 14, 24, 11, 83000),
 'prod_id': '68950008',
 'prod_name_txt': 'Women Plus Size Korean Fashion Plaid Sleeveless Woolen Vest Dress',
 'prod_price': 66.0,
 'prod_price_currency': 'MYR',
 'prod_ship_loc': 'MY',
 'prod_tags_txt': 'Women Clothing,Dresses,Mini',
 'seller_email': 'nonoplussize@hotmail.com',
 'seller_id': '55985302',
 'seller_join_ipv4': '147.158.152.194',
 'seller_list_ipv4': '220.158.201.20',
 'seller_loc_coarse': 'MY',
 'sent_datetime': datetime.datetime(2019, 6, 5, 14, 24, 8, 806000),
 'tag': 'Mini'}